# Trabalho Prático 1
## Bruno Jardim (a91680) José Ferreira (a91636)


### 2 - Sudoku 


2. Da definição do jogo “Sudoku” generalizado para a dimensão $N$; o problema tradicional corresponde ao caso $N=3$. O objetivo do Sudoku é preencher uma grelha de $,N^2\times N^2,$com inteiros positivos no intervalo $\,1$ até $\,N^2\,$, satisfazendo as seguintes regras:
    - Cada inteiro no intervalo $\,1$ até $\,N^2\,$ocorre  só uma vez em cada coluna, linha e secção $\,N\times N\,$.
    - No início do jogo uma fração $\,0\leq \alpha< 1\,$ das $\,N^4\,$ casas da grelha são preenchidas de forma consistente com a regra anterior.
         
#### Objetivo do trabalho:
   1. Construir um programa para inicializar a grelha a partir dos parâmetros $N$ e $\alpha$
   
   
   2. Construir soluções do problema para  as combinações de parâmetros $N\in\{3,4,5,6\}$  e$\,\alpha \in \{\,0.0\,,\,0.2\,,\,0.4\,,\,0.6\,\}$ . Que conclusões pode tirar da complexidade computacional destas soluções.


#### Análise do Problema:

   Existe um $N$ e um $\alpha$, onde $N^2 \times N^2$ corresponde ao tamanho do Sudoku que será representado e $\alpha$ à percentagem de células com um valor predefinido.
   
   Com estes valores iremos criar uma matriz que representa o Sudoku.
   
   Para solucionar o problema iremos representar este Sudoku como sendo um grafo e utilizaremos uma versão alterada da função $ip\_color$ desenvolvida nas aulas, com a finalidade desta colorir o grafo com $N^2$ cores
    
   Finalmente usaremos o grafo colorido para construir uma matriz correspondente ao Sudoku inicial resolvido.

#### Solução do Problema:
   1. Construir um Sudoku a partir dos parâmetros $N$ e $\alpha$
       - Para isto usaremos as funções $geraMat$ e $probCheck$, elas servem para criar uma matriz/Sudoku de tamanho $N^2$ com $\alpha$ probabilidade de conter um número em cada célula e para verificar probabilidades, respetivamente
           - Não existe garantia que o Sudoku gerado tenha solução

```python
    def geraMat(tam,prob):
        size = pow(tam,2)
        mat = [[0 for i in range(size)] for j in range(size)]
        for y in range(len(mat)):
            for x in range(len(mat[y])):
                if probCheck(prob):
                    num = random.randint(1, size)
                    if(validaPos(mat,x,y,num)):
                        mat[y][x] = num
        return mat
    
    
    def probCheck(prob):
    n = random.random()
    if(n <= prob):
        return True 
    return False 


```


2. Construir o grafo equivalente à matriz do Sudoku
    - Para isto usaremos as funções $createEdges$ e $checkSquare$, que serve para criar as edges do grafo
    


```python
def checkSquare(x1,y1,x2,y2,size):
        if x1//size == x2//size and y1//size == y2//size:
            return True
        return False

    
    
    
def createEdges(size):
    size = size**2
    mat = [x+1 for x in range(size**2)]
    mat = np.reshape(mat,(size,size)) 
    adj=[]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for outros in range(x,len(mat[y])):
                adj.append((mat[y][x],mat[y][outros]))
    for y in range(len(mat)):
        for x in range(len(mat[x])):
            for outros in range(y,len(mat)):
                adj.append((mat[y][x],mat[outros][x]))
            
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for yy in range(len(mat)):
                for xx in range(len(mat[yy])):
                    if checkSquare(x,y,xx,yy,int(math.sqrt(len(mat)))) and (mat[y][x],mat[yy][xx]) not in adj:
                        adj.append((mat[y][x],mat[yy][xx]))
   
    edges = [(x,y) for (x,y) in adj if x!=y and (y,x)]
    return edges




```

3. Criar o grafo a partir das edges
    - Para isto usaremos a biblioteca $NetworkX$
    

```python
grafo=nx.Graph(edges)
```


4. Colorir o grafo 
    - Para isto usaremos a função $ip\_color$ desenvolvida nas aulas, com algumas alterações. E a função $conv\_info$ para passar a informação da matriz para o grafo
    
    

```python
def conv_info(mat, graph): 
    for row in range( len(mat) ):
        for col in range( len(mat[row]) ):
            id_node = row*len(mat) + col + 1
            graph.nodes[id_node]['color'] = mat[row][col]
            
```

```python
def ip_color(graph,k):
    

    # criar solver
    solver = pywraplp.Solver('BOP', pywraplp.Solver.BOP_INTEGER_PROGRAMMING)
    #criar dicionario de variaveis x{i,j} 
    x = {}
    for i in graph:
        x[i] = {}
        for j in range(k):
            x[i][j] = solver.IntVar(0,1,'x[%i][%i]' % (i,j))

    # vertices pre-preenchidos 
    for o in graph:
        if graph.nodes[o]['color'] != 0: 
            c = graph.nodes[o]['color']
            solver.Add(x[o][c-1] == 1)

      
           
    
      
    # vertices adjacentes tem cores diferentes
    for o in graph:
        for d in graph[o]:
            for j in range(k):
                solver.Add(x[o][j] + x[d][j] <= 1)
    
            
                  
            
    
    # cada vertice adjacente tem cores diferentes

    for i in graph:
        solver.Add(sum([x[i][j] for j in range(k)]) == 1)  # ou solver.Add(sum(list(x[i].values())) == 1) 

    # invocar solver e colorir o grafo

    stat = solver.Solve()
    if stat == pywraplp.Solver.OPTIMAL:

      # colorir

        for i in graph:
            for j in range(k):
                if round(x[i][j].solution_value()) == 1:
                    graph.nodes[i]['color'] = j+1
        return True
    else:
        return False

```

5. Converter o grafo a matriz
    - Para isto usaremos a função $converte$, que recebe um grafo colorido e converte para uma matriz equivalente ao sudoku resolvido

```python
def converte(grafo, size):
    cores=[]
    for i in grafo:
        cores.append(grafo.nodes[i]['color'])
    solvedSudoku=np.reshape(cores,(int(math.sqrt(len(cores))),int(math.sqrt(len(cores)))))
    return solvedSudoku
```

## Funcões


In [17]:
!pip install ortools 

You should consider upgrading via the 'C:\Users\Bruno\anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
import numpy as np
import math 
import networkx as nx
import time
from random import randint
import random 
from ortools.linear_solver import pywraplp

In [1]:
def checkSquare(x1,y1,x2,y2,size):
        if x1//size == x2//size and y1//size == y2//size:
            return True
        return False

def createEdges(size):
    size = size**2
    mat = [x+1 for x in range(size**2)]
    mat = np.reshape(mat,(size,size)) 
    adj=[]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for outros in range(x,len(mat[y])):
                adj.append((mat[y][x],mat[y][outros]))
    for y in range(len(mat)):
        for x in range(len(mat[x])):
            for outros in range(y,len(mat)):
                adj.append((mat[y][x],mat[outros][x]))
            
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            for yy in range(len(mat)):
                for xx in range(len(mat[yy])):
                    if checkSquare(x,y,xx,yy,int(math.sqrt(len(mat)))) and (mat[y][x],mat[yy][xx]) not in adj:
                        adj.append((mat[y][x],mat[yy][xx]))
   
    edges = [(x,y) for (x,y) in adj if x!=y and (y,x)]
    return edges

In [2]:
def ip_color(graph,k):
    

    # criar solver
    solver = pywraplp.Solver('BOP', pywraplp.Solver.BOP_INTEGER_PROGRAMMING)
    #criar dicionario de variaveis x{i,j} 
    x = {}
    for i in graph:
        x[i] = {}
        for j in range(k):
            x[i][j] = solver.IntVar(0,1,'x[%i][%i]' % (i,j))

    # vertices pre-preenchidos 
    for o in graph:
        if graph.nodes[o]['color'] != 0: 
            c = graph.nodes[o]['color']
            solver.Add(x[o][c-1] == 1)

      
           
    
      
    # vertices adjacentes tem cores diferentes
    for o in graph:
        for d in graph[o]:
            for j in range(k):
                solver.Add(x[o][j] + x[d][j] <= 1)
    
            
                  
            
    
    # cada vertice adjacente tem cores diferentes

    for i in graph:
        solver.Add(sum([x[i][j] for j in range(k)]) == 1)  # ou solver.Add(sum(list(x[i].values())) == 1) 

    # invocar solver e colorir o grafo

    stat = solver.Solve()
    if stat == pywraplp.Solver.OPTIMAL:

      # colorir

        for i in graph:
            for j in range(k):
                if round(x[i][j].solution_value()) == 1:
                    graph.nodes[i]['color'] = j+1
        return True
    else:
        return False

In [3]:
def conv_info(mat, graph): 
    for row in range( len(mat) ):
        for col in range( len(mat[row]) ):
            id_node = row*len(mat) + col + 1
            graph.nodes[id_node]['color'] = mat[row][col]

In [4]:
def converte(grafo, size):
    cores=[]
    for i in grafo:
        cores.append(grafo.nodes[i]['color'])
    solvedSudoku=np.reshape(cores,(int(math.sqrt(len(cores))),int(math.sqrt(len(cores)))))
    return solvedSudoku

In [5]:
def probCheck(prob):
    n = random.random()
    if(n <= prob):
        return True 
    return False 

In [6]:
def validaPos(mat,x,y,n):
    tam = len(mat)
    for xver in range(tam):
        if mat[y][xver] == n:
            return False
    for yver in range(tam):
        if mat[yver][x] == n:
            return False
    sqSize = int(math.sqrt(tam))
    # Check Square
    inicioLin = y - y % sqSize
    inicioCol = x - x % sqSize
    for i in range(sqSize):
        for j in range(sqSize):
            if mat[i + inicioLin][j + inicioCol] == n:
                return False
    return True

def geraMat(tam,prob):
    size = pow(tam,2)
    mat = [[0 for i in range(size)] for j in range(size)]
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            if probCheck(prob):
                num = random.randint(1, size)
                if(validaPos(mat,x,y,num)):
                    mat[y][x] = num
    return mat

In [7]:
def checkZero(mat):
    for y in range( len(mat) ):
        for x in range( len(mat[y]) ):
            if mat[y][x]==0:
                return False
    return True

In [8]:
def display(mat):
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in mat]))
def sol():
    print()
    print('SOLUÇÃO:')
    print()

### Execução do código para n = 3.


In [27]:
edges=createEdges(3)
grafo5=nx.Graph(edges)
mat = geraMat(3, 0.2)

for x in mat: 
    print(x)
print('\n')

conv_info(mat, grafo5)
ti = time.perf_counter()
ip_color(grafo5,9)
tf = time.perf_counter()
solved = converte(grafo5, 9)
if checkZero(solved):
    display(solved)

else: print("Não há solução.")


print(f"Time {tf - ti: 0.5f} seconds")


[7, 6, 0, 0, 0, 0, 5, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 3, 0, 4, 0, 0, 0, 0]
[0, 9, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 8, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 7, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


   7   6   8   4   3   1   5   9   2
   3   2   9   8   6   5   1   7   4
   1   5   4   7   9   2   8   6   3
   8   7   3   5   4   6   2   1   9
   2   9   6   1   7   3   4   5   8
   4   1   5   2   8   9   6   3   7
   5   4   1   3   2   7   9   8   6
   6   3   2   9   1   8   7   4   5
   9   8   7   6   5   4   3   2   1
Time  0.25255 seconds


### Execução do código para n = 4.

In [28]:
edges=createEdges(4)
grafo5=nx.Graph(edges)
mat = geraMat(4, 0.2)

display(mat)
sol()
conv_info(mat, grafo5)
ti = time.perf_counter()
ip_color(grafo5,16)
tf = time.perf_counter()
display(converte(grafo5, 16))
print(f"Time {tf - ti: 0.5f} seconds")

   0   0  13   0   0   0   0   0   0  14   0   6   0   9   0   0
   0   0   0   0   0   0   0   0   0  15  10   0   0   8   0   0
   0   0   0   0   0   0   0   0   0   0   0   4   0   0   0  11
   0   0   0   0   0   0   0   0   0   0  13   0   0   4   0   0
   0   0   0   0   0   2   0   0   0   0   0   0   0   0  12   0
   0   0   3   0  11   0   0   0   0   9   0   0   0  13   0   0
   0   0   0  12   0   0   0   0   0   0   0   0   0   0   0   0
  10   0   0   0   0   0   0   0   0   0   1   0   0   7   0   0
   0   0  11   0   0  12   0   0   0   0   0   0   0  15   0   0
   0   0   0   5   0   7   0   0   0   0   0   0   0   0   9   0
   0   0   0   8   0  10   0   0   4   0   0   0  13   0   6   0
   0   0   0   0   0   0   3   0   0   0   0   2   0   0   0   0
   0   0   5   0   0   0   0  14   0   0   0   0  11   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0  15   0   0   0
   0   0   0  11   0  13   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  15   0 

### Execução do código para n = 5.

In [16]:
edges=createEdges(5)
grafo5=nx.Graph(edges)
mat = geraMat(5, 0.2)

display(mat)
sol()
conv_info(mat, grafo5)
ti = time.perf_counter()
ip_color(grafo5,25)
tf = time.perf_counter()
display(converte(grafo5, 25))
print(f"Time {tf - ti: 0.5f} seconds")

   0   0   0  14   0   0   0   0   0  21   0  11   0   0   0   0   0  12   0  15   0   0   0   0   0
   0   9   0   0  21   0   0   0   0   0   1   0   0   0   0   0  17   0   0   0   8   0   0   0   4
   0   0   0   0   0   0   0   0   0   4   6   0   0   0   0   0   7   0   3   1   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  19   0   0   0   3   0
   0   6   0   0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0   0   0   0   5   0   0
   9   0   0   0   0   0   1  14   0   0   7   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  22   0  10   0  13   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   3   0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   1   0   0   0  24  21   0   0   0   0   0  17   0   0  25   0   0   0   0   0   0   0   0  22   0
   0   0   0  12  15  11   0   0   0   0   0   0   0  19   0   8   0   9   0  25   0   0   

### Execução do código para n = 6.

In [30]:
edges=createEdges(6)
grafo5=nx.Graph(edges)
mat = geraMat(6, 0)

for x in mat: 
    print(x)
print('\n')

conv_info(mat, grafo5)
ti = time.perf_counter()
ip_color(grafo5,36)
tf = time.perf_counter()
print(converte(grafo5))
print(f"Time {tf - ti: 0.5f} seconds")

KeyboardInterrupt: 

## Versão Backtracking
 - Definimos também uma função de resolução de Sudokus com um algortimo com uma estratégia de Backtracking.
     - Apesar de ser mais eficiente em $N=3$, para valores de $N > 3$ o tempo de execução não é viável
     

In [12]:
def valido(mat,x,y,n):
    tam = len(mat)
    for xver in range(tam):
        if mat[y][xver] == n:
            return False
    for yver in range(tam):
        if mat[yver][x] == n:
            return False




    sqSize=int(math.sqrt(tam))
    #Check Square
    inicioLin=y - y % sqSize
    inicioCol=x - x % sqSize



    for i in range(sqSize):
        for j in  range(sqSize):
            if mat[i+inicioLin][j+inicioCol] == n:
                return False
    return True




def sudokuSolver(mat, x, y):
    fim = len(mat)
    #Verificamos se chegamos ao final da matriz, se sim encontramos uma solucao

    if y == fim-1 and x == fim:
        return True
    #Se chegarmos ao final da linha voltamos ao inicio na linha seguinte
    if x == fim:
        y+=1
        x=0
    #Se o numero estiver preenchido saltamos a frente pois é um dos pre-preenchidos
    if mat[y][x] > 0 :
        return sudokuSolver(mat,x+1,y)
    #Tentamos arranjar solucao para o sudoku
    for n in range(1,fim+1):
        #Verificamos se o numero que tentamos preencher e valido
        if valido(mat,x,y,n):
            #Preenchemos com o numero caso tenha sido validado
            mat[y][x] = n
            #Verificamos se e possivel prosseguir, caso contrario damos backtrack
            if sudokuSolver(mat,x+1,y):
                return True
        mat[y][x] = 0
    return False

In [15]:
mat=geraMat(3,0.3)
display(mat)
sol()
ti = time.perf_counter()
sudokuSolver(mat,0,0)
tf = time.perf_counter()
display(mat)
print(f"Time {tf - ti: 0.5f} seconds")

   6   0   0   0   0   0   0   0   0
   0   8   0   0   0   0   0   0   0
   0   7   4   0   0   0   1   3   0
   0   0   5   0   0   0   0   0   1
   0   0   0   0   4   0   0   0   0
   0   0   0   0   8   0   0   0   0
   0   0   0   0   0   0   0   0   8
   9   6   0   0   0   7   5   0   0
   0   0   3   1   0   0   7   0   0

SOLUÇÃO:

   6   1   2   3   5   4   8   7   9
   3   8   9   7   1   2   4   5   6
   5   7   4   8   6   9   1   3   2
   4   2   5   6   7   3   9   8   1
   8   3   1   9   4   5   6   2   7
   7   9   6   2   8   1   3   4   5
   1   4   7   5   3   6   2   9   8
   9   6   8   4   2   7   5   1   3
   2   5   3   1   9   8   7   6   4
Time  4.29960 seconds


## Gráficos


1. Comparação dos diferentes solvers para $N = 3$
![alt text](Sudoku_n3.png "Comparação N=3")

![alt text](Sudoku_BOP_BT.png "Comparação N=3")

2. Comparação do solver $BOP$ para $N \in [4,5]$
![alt text](Sudoku_n4n5.png "Comparação N=3")